13) this version is 7*7 game with positional property, values, centered\edged for each node. The hyperparameters are 20000,1000,0.3 for clauses,T and S respectively.----------Max Acuraccy is 95% on test data.

In [1]:

from GraphTsetlinMachine.graphs import Graphs
from GraphTsetlinMachine.tm import MultiClassGraphTsetlinMachine
from time import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [2]:
import os
current_directory = os.getcwd()
print(current_directory)
dir_base = os.path.join(current_directory,'Hex_game_result_7_kaggle.csv')
print(dir_base)
#des_base = os.path.join(dir_base,'3_3_small.csv')
#print(des_base)

/home/dominikaik/ls-prosjekt
/home/dominikaik/ls-prosjekt/Hex_game_result_7_kaggle.csv


In [4]:
df = pd.read_csv('Hex_game_result_7_kaggle.csv')
df = df.iloc[:10000]
print('done')

done


In [5]:
input_matrix = df.iloc[:, :-1].values.reshape(-1, 7, 7)  # Reshape to (num_games, 7, 7)
input_matrix = np.where(input_matrix == -1, 2, input_matrix)  # Replace -1 with 2

df['winner'] = df.iloc[:, -1]  # Last column as the winner
# Replace -1 with 0 in the 'winner' column if necessary
df['winner'] = df['winner'].replace(-1, 0)
type(input_matrix[0])

numpy.ndarray

In [6]:
df['winner']

0       1
1       1
2       0
3       0
4       1
       ..
9995    0
9996    1
9997    0
9998    1
9999    0
Name: winner, Length: 10000, dtype: int64

In [7]:
df.shape[0]

10000

In [8]:
# Split into training and test sets
subset_size = int(df.shape[0] * 0.9)
X_train = input_matrix[:subset_size]
X_test = input_matrix[subset_size:]
Y_train = df['winner'].iloc[:subset_size].values
Y_test = df['winner'].iloc[subset_size:].values

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(type(Y_train))
print(type(X_train))
print(type(Y_test))

X_train shape: (9000, 7, 7)
X_test shape: (1000, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [9]:
Y_train

array([1, 1, 0, ..., 0, 0, 1])

In [10]:
# Initialize a dictionary to store the number of edges for each node
node_edges = {}
dim=7
for row in range(dim):
    for col in range(dim):
        node_id = row * dim + col  # Unique node ID for each hexagon

        # Dynamically calculate the number of neighbors for each node
        number_of_neighbors = 0
        if row > 0: number_of_neighbors += 1  # Above
        if row < dim - 1: number_of_neighbors += 1  # Below
        if col > 0: number_of_neighbors += 1  # Left
        if col < dim - 1: number_of_neighbors += 1  # Right
        if row > 0 and col < dim - 1: number_of_neighbors += 1  # AboveRight
        if row < dim - 1 and col > 0: number_of_neighbors += 1  # BelowLeft
        
        # Store the number of edges for this node
        node_edges[node_id] = number_of_neighbors

# Extract just the values (number of edges) into a list
n_edges_list = list(node_edges.values())

# Output the list of edge values (optional)
print(n_edges_list)
sum_elements = sum(n_edges_list)
sum_elements

[2, 4, 4, 4, 4, 4, 3, 4, 6, 6, 6, 6, 6, 4, 4, 6, 6, 6, 6, 6, 4, 4, 6, 6, 6, 6, 6, 4, 4, 6, 6, 6, 6, 6, 4, 4, 6, 6, 6, 6, 6, 4, 3, 4, 4, 4, 4, 4, 2]


240

In [11]:
#this part is responsible for generating edges (connections) between adjacent cells in a 3x3 Hex grid. Rows, columns and Diagonals
import itertools
dim=7
# Function to generate edges for a 3x3 grid
def generate_edges():
    edges = []

    # Row connections (horizontal)
    for row in range(dim):
        for col in range(dim-1):  # 2 because we connect (col, col+1)
            edges.append(((row, col), (row, col+1)))
    
    # Column connections (vertical)
    for col in range(dim):
        for row in range(dim-1):  # 2 because we connect (row, row+1)
            edges.append(((row, col), (row+1, col)))

    # Diagonal connections (from bottom-left to top-right)
    for row in range(dim-1):
        for col in range(dim-1):  # Diagonals will only connect within the bounds
            edges.append(((row, col+1), (row+1, col)))

    return edges

# Generate edges
edges = generate_edges()

edges
print(len(edges))


120


#########################################################################################
##############################################################################################

In [12]:
import argparse
import sys

def default_args(**kwargs):
    parser = argparse.ArgumentParser()
    parser.add_argument("--epochs", default=100, type=int)
    parser.add_argument("--number-of-clauses", default=20000, type=int)
    parser.add_argument("--T", default=1000, type=int)
    parser.add_argument("--s", default=0.3, type=float)
    parser.add_argument("--depth", default=3, type=int)
    parser.add_argument("--hypervector-size", default=512, type=int)
    parser.add_argument("--hypervector-bits", default=2, type=int)
    parser.add_argument("--message-size", default=512, type=int)
    parser.add_argument("--message-bits", default=2, type=int)
    parser.add_argument('--double-hashing', dest='double_hashing', default=False, action='store_true')
    parser.add_argument("--max-included-literals", default=16, type=int)

    # Add this line to avoid Jupyter arguments causing an error
    args = parser.parse_args(args=[])  # Ignore any command line arguments

    for key, value in kwargs.items():
        if key in args.__dict__:
            setattr(args, key, value)
    return args
args = default_args()

In [13]:
from GraphTsetlinMachine.graphs import Graphs
import numpy as np
from GraphTsetlinMachine.tm import MultiClassGraphTsetlinMachine
from time import time


dim = 7

number_of_nodes = dim * dim


symbol_names = ['Red','Blue','White']

#for dist in range(dim // 2 + 1):
    #symbol_names.append(f'DistToWall_{dist}')

# Adjacency to walls
#symbol_names.extend(['TouchingRedWall', 'TouchingBlueWall'])

# Row and column information
for row in range(dim):
    symbol_names.append(f'Row_{row}')
for col in range(dim):
    symbol_names.append(f'Col_{col}')


symbol_names.extend(['CenterAligned', 'EdgeAligned'])

symbol_names

['Red',
 'Blue',
 'White',
 'Row_0',
 'Row_1',
 'Row_2',
 'Row_3',
 'Row_4',
 'Row_5',
 'Row_6',
 'Col_0',
 'Col_1',
 'Col_2',
 'Col_3',
 'Col_4',
 'Col_5',
 'Col_6',
 'CenterAligned',
 'EdgeAligned']

In [14]:
# Initialize the graph structure for training
graphs_train = Graphs(
    number_of_graphs=subset_size,
    symbols=symbol_names,
    hypervector_size=args.hypervector_size,
    hypervector_bits=args.hypervector_bits,
    double_hashing=args.double_hashing,
)
# Prepare nodes
for graph_id in range(X_train.shape[0]):
    graphs_train.set_number_of_graph_nodes(
        graph_id=graph_id,
        number_of_graph_nodes=dim**2,
    )
graphs_train.prepare_node_configuration()

# Prepare edges
for graph_id in range(X_train.shape[0]):
    for k in range(dim**2):
        graphs_train.add_graph_node(graph_id, k, n_edges_list[k])
graphs_train.prepare_edge_configuration()


# Add nodes and their properties for each graph
dim = 7  # For a 3x3 grid
for graph_id in range(X_train.shape[0]):
    for k in range(dim * dim):  # 9 nodes for the 3x3 grid
        # Get the value of the current cell in the board
        cell_value = X_train[graph_id].flatten()[k]  # Flatten to 1D array

        # Convert node index to (row, col) coordinates
        row, col = divmod(k, dim)
        
        # Feature 1: Assign player color
        if cell_value == 1:  # Player 1
            graphs_train.add_graph_node_property(graph_id, k, "Blue")
        elif cell_value == 2:  # Player 2
            graphs_train.add_graph_node_property(graph_id, k, "Red")
        else:  # Empty cell
            graphs_train.add_graph_node_property(graph_id, k, "White")

        # Feature 2: Distance to walls
        #distance_to_wall = min(row, col, dim - row - 1, dim - col - 1)
        #graphs_train.add_graph_node_property(graph_id, k, f'DistToWall_{distance_to_wall}')

        # Feature 3: Adjacency to walls
        #if row == 0 or row == dim - 1:
            #graphs_train.add_graph_node_property(graph_id, k, "TouchingRedWall")
        #if col == 0 or col == dim - 1:
            #graphs_train.add_graph_node_property(graph_id, k, "TouchingBlueWall")
        # Feature 6: Center or Border Alignment
        center_dist = abs(row - dim // 2) + abs(col - dim // 2)
        alignment = "CenterAligned" if center_dist <= dim // 2 else "EdgeAligned"
        graphs_train.add_graph_node_property(graph_id, k, alignment)

        # Feature 4: Row and column information
        graphs_train.add_graph_node_property(graph_id, k, f'Row_{row}')
        graphs_train.add_graph_node_property(graph_id, k, f'Col_{col}')


# Prepare edge configuration for the training data
graphs_train.prepare_edge_configuration()

# Add edge connections based on the predefined edges
for graph_id in range(X_train.shape[0]):
    for edge in edges:
        node_id = edge[0][0] * dim + edge[0][1]  # Convert (row, col) to node ID
        destination_node_id = edge[1][0] * dim + edge[1][1]

        # Add bidirectional edges
        graphs_train.add_graph_node_edge(graph_id, node_id, destination_node_id, edge_type_name=0)
        graphs_train.add_graph_node_edge(graph_id, destination_node_id, node_id, edge_type_name=0)

# Encode the graph
graphs_train.encode()
print('end')

end


In [15]:

# Initialize the graph structure for test data
graphs_test = Graphs(X_test.shape[0], init_with=graphs_train)


# Prepare nodes
for graph_id in range(X_test.shape[0]):
    graphs_test.set_number_of_graph_nodes(
        graph_id=graph_id,
        number_of_graph_nodes=dim**2,
    )
graphs_test.prepare_node_configuration()

# Prepare edges
for graph_id in range(X_test.shape[0]):
    for k in range(dim**2):
        graphs_test.add_graph_node(graph_id, k, n_edges_list[k])
graphs_test.prepare_edge_configuration()


# Add nodes and their properties for each graph
dim = 7  # For a 3x3 grid
for graph_id in range(X_test.shape[0]):
    for k in range(dim * dim):  # 9 nodes for the 3x3 grid
        # Get the value of the current cell in the board
        cell_value = X_test[graph_id].flatten()[k]  # Flatten to 1D array

        # Convert node index to (row, col) coordinates
        row, col = divmod(k, dim)
        
        # Feature 1: Assign player color
        if cell_value == 1:  # Player 1
            graphs_test.add_graph_node_property(graph_id, k, "Blue")
        elif cell_value == 2:  # Player 2
            graphs_test.add_graph_node_property(graph_id, k, "Red")
        else:  # Empty cell
            graphs_test.add_graph_node_property(graph_id, k, "White")

        # Feature 2: Distance to walls
        #distance_to_wall = min(row, col, dim - row - 1, dim - col - 1)
        #graphs_test.add_graph_node_property(graph_id, k, f'DistToWall_{distance_to_wall}')

        # Feature 3: Adjacency to walls
        #if row == 0 or row == dim - 1:
            #graphs_test.add_graph_node_property(graph_id, k, "TouchingRedWall")
        #if col == 0 or col == dim - 1:
            #graphs_test.add_graph_node_property(graph_id, k, "TouchingBlueWall")

        # Feature 6: Center or Border Alignment
        center_dist = abs(row - dim // 2) + abs(col - dim // 2)
        alignment = "CenterAligned" if center_dist <= dim // 2 else "EdgeAligned"
        graphs_test.add_graph_node_property(graph_id, k, alignment)


        # Feature 4: Row and column information
        graphs_test.add_graph_node_property(graph_id, k, f'Row_{row}')
        graphs_test.add_graph_node_property(graph_id, k, f'Col_{col}')


#

# Add edge connections based on the predefined edges
for graph_id in range(X_test.shape[0]):
    for edge in edges:
        node_id = edge[0][0] * dim + edge[0][1]  # Convert (row, col) to node ID
        destination_node_id = edge[1][0] * dim + edge[1][1]

        # Add bidirectional edges
        graphs_test.add_graph_node_edge(graph_id, node_id, destination_node_id, edge_type_name=0)
        graphs_test.add_graph_node_edge(graph_id, destination_node_id, node_id, edge_type_name=0)
graphs_test.encode()
print('end')


end


In [16]:
# Train the Tsetlin Machine
tm = MultiClassGraphTsetlinMachine(
    args.number_of_clauses,
    args.T,
    args.s,
    depth=args.depth,
    message_size=args.message_size,
    message_bits=args.message_bits,
    max_included_literals=args.max_included_literals,
    grid=(16*13,1,1),
    block=(128,1,1)
)

start_training = time()
for i in range(args.epochs):
    tm.fit(graphs_train, Y_train, epochs=1, incremental=True)
    print(f"Epoch#{i+1} -- Accuracy train: {np.mean(Y_train == tm.predict(graphs_train))}", end=' ')
    print(f"-- Accuracy test: {np.mean(Y_test == tm.predict(graphs_test))} ")
stop_training = time()
print(f"Time: {stop_training - start_training}")

Initialization of sparse structure.
Epoch#1 -- Accuracy train: 0.7197777777777777 -- Accuracy test: 0.719 
Epoch#2 -- Accuracy train: 0.7537777777777778 -- Accuracy test: 0.737 
Epoch#3 -- Accuracy train: 0.7341111111111112 -- Accuracy test: 0.745 
Epoch#4 -- Accuracy train: 0.6885555555555556 -- Accuracy test: 0.7 
Epoch#5 -- Accuracy train: 0.8233333333333334 -- Accuracy test: 0.784 
Epoch#6 -- Accuracy train: 0.8182222222222222 -- Accuracy test: 0.773 
Epoch#7 -- Accuracy train: 0.8331111111111111 -- Accuracy test: 0.798 
Epoch#8 -- Accuracy train: 0.7448888888888889 -- Accuracy test: 0.701 
Epoch#9 -- Accuracy train: 0.8455555555555555 -- Accuracy test: 0.806 
Epoch#10 -- Accuracy train: 0.8278888888888889 -- Accuracy test: 0.782 
Epoch#11 -- Accuracy train: 0.8491111111111111 -- Accuracy test: 0.794 
Epoch#12 -- Accuracy train: 0.824 -- Accuracy test: 0.808 
Epoch#13 -- Accuracy train: 0.8578888888888889 -- Accuracy test: 0.806 
Epoch#14 -- Accuracy train: 0.8548888888888889 -- Ac

#####################################################################################################
##################################another features###################################################